### FINA 4380 with Marius Popescu

## Portfolio Value, Return and Risk

In [1]:
import numpy as np
import pandas as pd

import pandas_datareader.data as web
import datetime as dt

In [9]:
data_all = pd.read_csv('month_rets.csv',
                    usecols = ['date','TICKER', 'PRC', 'CFACPR'],
                    index_col = 'date',
                    parse_dates = True)
data_all['ADJ_PRC'] = data_all['PRC']/data_all['CFACPR']
data_all.index.name = 'Date'
data_all.head()

,TICKER,PRC,CFACPR,ADJ_PRC
Date,,,,
2013-01-31,MSFT,27.450,1.0,27.450
2013-02-28,MSFT,27.800,1.0,27.800
2013-03-28,MSFT,28.605,1.0,28.605
2013-04-30,MSFT,33.100,1.0,33.100
2013-05-31,MSFT,34.900,1.0,34.900


In [10]:
prices = pd.DataFrame()
for ticker in data_all['TICKER'].unique():
    prices[ticker] = data_all[data_all['TICKER']==ticker]['ADJ_PRC']
prices.head()

,MSFT,IBM,FB,AAPL,AMZN,GOOGL
Date,,,,,,
2013-01-31,27.450,203.07001,30.981,65.069999,265.50000,378.431569
2013-02-28,27.800,200.83000,27.250,63.057141,264.26999,401.221899
2013-03-28,28.605,213.30000,25.580,63.237143,266.48999,397.710446
2013-04-30,33.100,202.53999,27.769,63.254286,253.81000,412.925039
2013-05-31,34.900,208.02000,24.348,64.247856,269.20001,436.285252


### Investment Value Over Time

The value of an initial $1 investment can be found by dividing an asset's current price by the asset's initial price.

In [11]:
inv_value = pd.DataFrame()
for column in prices:
    inv_value[column] = prices[column]/prices[column].iloc[0]
inv_value.head()

,MSFT,IBM,FB,AAPL,AMZN,GOOGL
Date,,,,,,
2013-01-31,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2013-02-28,1.012750,0.988969,0.879571,0.969066,0.995367,1.060223
2013-03-28,1.042077,1.050377,0.825667,0.971833,1.003729,1.050944
2013-04-30,1.205829,0.997390,0.896324,0.972096,0.955970,1.091149
2013-05-31,1.271403,1.024376,0.785901,0.987365,1.013936,1.152878


### Portfolio Value and Return

#### A. Regular Rebalancing

A portfolio that is rebalanced every period (daily, monthly etc.) is a portfolio whose asset allocations are adjusted to target weights at the beginning of the period (daily, monthly etc.)

In [12]:
returns=prices.pct_change()
returns.head()

,MSFT,IBM,FB,AAPL,AMZN,GOOGL
Date,,,,,,
2013-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2013-02-28,0.012750,-0.011031,-0.120429,-0.030934,-0.004633,0.060223
2013-03-28,0.028957,0.062092,-0.061284,0.002855,0.008400,-0.008752
2013-04-30,0.157140,-0.050445,0.085575,0.000271,-0.047581,0.038255
2013-05-31,0.054381,0.027056,-0.123195,0.015708,0.060636,0.056573


#### Generating Random Weights

We can use the Standard Uniform distribution to generate random numbers between 0 and 1, and then converting the numbers to potential weights by making sure that their sum is 1.

In [13]:
np.random.seed(1000)
weights = np.random.uniform(size=len(returns.columns))
weights = weights / np.sum(weights)
print(weights)

[0.19890866 0.03500037 0.28920211 0.14674659 0.2655225  0.06461977]


#### For a Regularly Rebalanced portfolio, the return is calculated as the Dot product of the cross-sectional return vector and the weights vector.

In [14]:
MReb_portfolio = pd.DataFrame({'MReb_Ret':np.dot(returns,weights)},index = returns.index)
MReb_portfolio.head()

,MReb_Ret
Date,
2013-01-31,NaN
2013-02-28,-0.034556
2013-03-28,-0.007707
2013-04-30,0.044117
2013-05-31,-0.001804


#### For a Regularly Rebalanced portfolio, portfolio value is calculated as the cumulative product of (1 + period return).

In [15]:
MReb_portfolio['MReb_Port_Val'] = MReb_portfolio[['MReb_Ret']].apply(lambda x:x.add(1).cumprod())
MReb_portfolio['MReb_Port_Val'].iloc[0]=1

In [16]:
MReb_portfolio.head()

,MReb_Ret,MReb_Port_Val
Date,,
2013-01-31,NaN,1.000000
2013-02-28,-0.034556,0.965444
2013-03-28,-0.007707,0.958004
2013-04-30,0.044117,1.000268
2013-05-31,-0.001804,0.998464


#### Portfolio Rebalancing Example

#### B. No Rebalancing - Buy and Hold Strategy

#### For a Buy and Hold strategy, portfolio value is calculated as the Dot product of the cross-sectional asset values and the weights vector.

In [17]:
BH_portfolio = pd.DataFrame({'BH_Port_Val': np.dot(inv_value, weights)},index = inv_value.index)
BH_portfolio.head()

,BH_Port_Val
Date,
2013-01-31,1.000000
2013-02-28,0.965444
2013-03-28,0.959864
2013-04-30,1.000971
2013-05-31,1.004645


#### For a Buy and Hold strategy, portfolio return is calculated as the change in the value of the portfolio.

In [18]:
BH_portfolio['BH_Ret'] = BH_portfolio['BH_Port_Val'].pct_change()
BH_portfolio.head()

,BH_Port_Val,BH_Ret
Date,,
2013-01-31,1.000000,NaN
2013-02-28,0.965444,-0.034556
2013-03-28,0.959864,-0.005780
2013-04-30,1.000971,0.042826
2013-05-31,1.004645,0.003671


In [21]:
port_ret = pd.merge(MReb_portfolio,BH_portfolio,on='Date')
port_ret.head()

,MReb_Ret,MReb_Port_Val,BH_Port_Val,BH_Ret
Date,,,,
2013-01-31,NaN,1.000000,1.000000,NaN
2013-02-28,-0.034556,0.965444,0.965444,-0.034556
2013-03-28,-0.007707,0.958004,0.959864,-0.005780
2013-04-30,0.044117,1.000268,1.000971,0.042826
2013-05-31,-0.001804,0.998464,1.004645,0.003671


#### Q1: Which of the two portfolios performed the best over the entire sample period? What was the total return?

#### Q2: Which of the two portfolios performed the worst over the entire sample period? What was the total return?

### Portfolio Risk

#### Method 1

In [22]:
# Monthly Portfolio Risk (Standard Deviation)
port_ret[['MReb_Ret','BH_Ret']].std()

MReb_Ret    0.047619
BH_Ret      0.045852
dtype: float64

In [23]:
# Annualized Portfolio Risk (Standard Deviation)
port_ret[['MReb_Ret','BH_Ret']].std()*np.sqrt(12)

MReb_Ret    0.164957
BH_Ret      0.158835
dtype: float64

#### Method 2 - Only for Regularly Rebalanced Portfolios

In [24]:
# The Variance-Covariance Matrix
returns.cov()

,MSFT,IBM,FB,AAPL,AMZN,GOOGL
MSFT,0.003722,0.000752,-0.000218,0.001300,0.001437,0.001566
IBM,0.000752,0.002489,-0.000056,0.001057,0.000798,0.000151
FB,-0.000218,-0.000056,0.008298,0.001263,0.001414,0.001275
AAPL,0.001300,0.001057,0.001263,0.004347,0.001732,0.000971
AMZN,0.001437,0.000798,0.001414,0.001732,0.005964,0.002675
GOOGL,0.001566,0.000151,0.001275,0.000971,0.002675,0.003184


In [25]:
# Monthly Portfolio Risk (Standard Deviation)
MReb_port_risk = np.sqrt(np.dot(weights, np.dot(returns.cov(), weights)))
MReb_port_risk

0.04761912478403526

In [49]:
#MReb_port_risk = np.dot(weights, np.dot(returns.cov(), weights))**0.5
#MReb_port_risk

In [26]:
# Annualized Portfolio Risk (Standard Deviation)
MReb_port_risk*np.sqrt(12)

0.1649574870758228

### Sharpe Ratio

In [27]:
# Download the risk-free rate from Ken French's website
start = dt.datetime(1927, 1, 1)
end = dt.datetime.now()

#three_factors = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0]
rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']]
rf.head()

,RF
Date,
1927-01,0.25
1927-02,0.26
1927-03,0.30
1927-04,0.25
1927-05,0.30


In [29]:
# We convert the Period Index to a Datetime Index
rf.index = rf.index.to_timestamp(how='end').normalize()

In [30]:
rf = rf.apply(lambda x: x/100)
rf.head()

,RF
Date,
1927-01-31,0.0025
1927-02-28,0.0026
1927-03-31,0.0030
1927-04-30,0.0025
1927-05-31,0.0030


In [55]:
port_ret.head()

,MReb_Ret,MReb_Port_Val,BH_Port_Val,BH_Ret
Date,,,,
2013-01-31,NaN,1.000000,1.000000,NaN
2013-02-28,-0.034556,0.965444,0.965444,-0.034556
2013-03-28,-0.007707,0.958004,0.959864,-0.005780
2013-04-30,0.044117,1.000268,1.000971,0.042826
2013-05-31,-0.001804,0.998464,1.004645,0.003671


In [31]:
#Change each index value to the end of the month
port_ret.index = port_ret.index + pd.offsets.MonthEnd(0)

In [32]:
data = pd.merge(port_ret.drop(['MReb_Port_Val','BH_Port_Val'],axis=1), rf, on = 'Date')
data.head()

,MReb_Ret,BH_Ret,RF
Date,,,
2013-01-31,NaN,NaN,0.0
2013-02-28,-0.034556,-0.034556,0.0
2013-03-31,-0.007707,-0.005780,0.0
2013-04-30,0.044117,0.042826,0.0
2013-05-31,-0.001804,0.003671,0.0


In [33]:
data['Ex_MReb_Ret'] = data['MReb_Ret'] - data['RF']
data['Ex_BH_Ret'] = data['BH_Ret'] - data['RF']
data.head()

,MReb_Ret,BH_Ret,RF,Ex_MReb_Ret,Ex_BH_Ret
Date,,,,,
2013-01-31,NaN,NaN,0.0,NaN,NaN
2013-02-28,-0.034556,-0.034556,0.0,-0.034556,-0.034556
2013-03-31,-0.007707,-0.005780,0.0,-0.007707,-0.005780
2013-04-30,0.044117,0.042826,0.0,0.044117,0.042826
2013-05-31,-0.001804,0.003671,0.0,-0.001804,0.003671


In [34]:
SR_MReb_port = (data['Ex_MReb_Ret'].mean()*12)/(data['Ex_MReb_Ret'].std()*np.sqrt(12))
SR_MReb_port

1.8239767901387445

In [35]:
SR_BH_port = (data['Ex_BH_Ret'].mean()*12)/(data['Ex_BH_Ret'].std()*np.sqrt(12))
SR_BH_port

1.8667407532048244